In [1]:
import os
import numpy as np
import torch
import parmed

import mymd

In [2]:
data_path = '../../data/md_from_scratch/'

old_pdb_file = os.path.join(data_path, 'ala15.pdb')
new_pdb_file = os.path.join(data_path, 'ala15_new.pdb')
psf_file = os.path.join(data_path, 'ala15.psf')
prmtop_file = os.path.join(data_path, 'ala15.prmtop')

In [3]:
# mymd.gen_files_from_pdb(old_pdb_file=old_pdb_file,
#                         psf_file=psf_file,
#                         prmtop_file=prmtop_file,
#                         new_pdb_file=new_pdb_file)

In [4]:
mol = mymd.get_molecule(prmtop_file=prmtop_file, pdb_file=new_pdb_file)
print('---coords---')
print(mol.coords[:2])
print('-----box-----')
print(mol.box)
print('---charge---')
print(mol.charge[:3])
print('---masses---')
print(mol.masses[:3])
print('----bonds----')
print(mol.bonds[:4])
print('---bonds.shape---')
print(mol.bonds.shape)

2023-05-06 14:57:59,312 - numexpr.utils - INFO - NumExpr defaulting to 8 threads.


---coords---
[[[ 3.326]
  [ 1.548]
  [-0.   ]]

 [[ 4.046]
  [ 0.84 ]
  [-0.   ]]]
-----box-----
[[0.]
 [0.]
 [0.]]
---charge---
[0.1414 0.1997 0.1997]
---masses---
[14.00672   1.007947  1.007947]
----bonds----
[[4 5]
 [6 7]
 [6 8]
 [6 9]]
---bonds.shape---
(152, 2)


In [5]:
ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=True)
print(ff.params.bond_types[('C1', 'N1')], end='\n\n')

struct = parmed.amber.AmberParm(prmtop_file)
for bond in struct.bonds:
    key = (bond.atom1.type, bond.atom2.type)
    if key[0] == 'C1' and key[1] == 'N1' or key[0] == 'N1' and key[1] == 'C1':
        print(bond.type)

print()
print(ff.mol.atomtype[:10])
print(ff.mol.charge[:10])  # same atomtype can have different charge ('C1')
print(set(ff.mol.atomtype))
print(set(ff.params.atom_types))

<BondType; k=367.000, req=1.471>

<BondType; k=367.000, req=1.471>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>
<BondType; k=337.000, req=1.449>

['N1' 'H1' 'H1' 'H1' 'C1' 'H2' 'C1' 'H3' 'H3' 'H3']
[ 0.1414  0.1997  0.1997  0.1997  0.0962  0.0889 -0.0597  0.03    0.03
  0.03  ]
{'O1', 'H1', 'C1', 'H3', 'H4', 'N1', 'H2', 'C2'}
{'O1', 'H1', 'C1', 'H3', 'H4', 'N1', 'H2', 'C2'}


In [6]:
system = mymd.System(mol, ff)
system.set_device_and_dtype('cpu', torch.float64)
print(system.bond_params[:5])
print(system.improper_params[0]['params'][:5])
print(system.uni_atom_types == np.unique(mol.atomtype))
print(system.uni_atom_types[system.mapped_atom_types[:6]] == mol.atomtype[:6])
print(system.box)

tensor([[434.0000,   1.0100],
        [434.0000,   1.0100],
        [434.0000,   1.0100],
        [367.0000,   1.4710],
        [340.0000,   1.0900]], dtype=torch.float64)
tensor([[10.5000,  3.1416,  2.0000],
        [ 1.1000,  3.1416,  2.0000],
        [10.5000,  3.1416,  2.0000],
        [ 1.1000,  3.1416,  2.0000],
        [10.5000,  3.1416,  2.0000]], dtype=torch.float64)
[ True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True]
None


In [7]:
prmtop_file = os.path.join(data_path, 'structure.prmtop')
coor_file = os.path.join(data_path, 'input.coor')
xsc_file = os.path.join(data_path, 'input.xsc')
mol = mymd.get_molecule(prmtop_file=prmtop_file, coor_file=coor_file, xsc_file=xsc_file)
ff = mymd.PrmtopMolForceField(mol, prmtop_file, allow_unequal_duplicates=False)
system = mymd.System(mol, ff)
system.set_device_and_dtype('cpu', torch.float64)

In [8]:
print(system.bond_params[:5])
print(system.improper_params[0]['params'][:5])
print(system.uni_atom_types == np.unique(mol.atomtype))
print(system.uni_atom_types[system.mapped_atom_types[:6]] == mol.atomtype[:6])
print(system.box, mol.box)

tensor([[340.0000,   1.0900],
        [340.0000,   1.0900],
        [340.0000,   1.0900],
        [317.0000,   1.5220],
        [570.0000,   1.2290]], dtype=torch.float64)
tensor([[10.5000,  3.1416,  2.0000],
        [ 1.1000,  3.1416,  2.0000],
        [10.5000,  3.1416,  2.0000],
        [ 1.1000,  3.1416,  2.0000]], dtype=torch.float64)
[ True  True  True  True  True  True  True  True  True  True]
[ True  True  True  True  True  True]
tensor([19.8388, 19.6193, 19.6342], dtype=torch.float64) [[19.83881]
 [19.6193 ]
 [19.6342 ]]
